In [1]:
import pandas as pd 
import json
import random
import time
import hashlib

In [2]:
cities = ['Los Ángeles', 'Concepción', 'Santiago']
payment_online = ['Credit_card', 'Debit_card']
payment_store = ['Cash', 'Credit_card', 'Debit_card']
source = ['Online', 'On-site']
status_purchase = ['COMPLETED','FAILED_CHECKOUT','FAILED_API_RESPONSE','INSUFICCIENT_FUNDS','COMPLETED','COMPLETED','COMPLETED','COMPLETED','COMPLETED','COMPLETED','FAILED_API_RESPONSE','INSUFICCIENT_FUNDS','USER_ERROR','FRAUD','COMPLETED','COMPLETED','COMPLETED']

In [3]:
la_city = [(-37.46827078074566, -72.35255427015483)]
conce_city = [(-36.82574208790205, -73.0484756767045)]
stgo_city = [(-33.41811053129037, -70.6069033367)]

In [4]:
random.choice(payment_store)

'Debit_card'

In [5]:
def get_pay_method(source, status_purchase, payment_online, payment_store):
    if source == 'On-site':
        payment = random.choice(payment_store)
        status = 'COMPLETED'
        order_type = 'STORE'
    elif source == 'Online':
        payment = random.choice(payment_online)
        status = random.choice(status_purchase)
        order_type = 'ONLINE'
    return payment, status, order_type

def get_coords(city):
    if city == 'Los Ángeles':
        coords = random.choice(la_city)
    elif city == 'Concepción':
        coords = random.choice(conce_city)
    elif city == 'Santiago':
        coords = random.choice(stgo_city)
    return coords

In [6]:
get_pay_method('On-site',status_purchase,payment_online,payment_store)

('Credit_card', 'COMPLETED', 'STORE')

In [12]:
city = random.choice(cities)
coords = get_coords(city)
print(f'{city} | {coords}')

Los Ángeles | (-37.46827078074566, -72.35255427015483)


In [13]:
df = pd.read_excel('products.xlsx')
df.head(5)

,PRODUCT_NAME,PRICING,BRAND,CATEGORY
0,LED 50” AU7090 4K UHD Smart TV 2022,309990,Samsung,TV y Video
1,Smart Watch GT 4 46MM Negro,189990,Huawei,Wearables
2,Cámara Reflex T7+Lente 18-55 Is,549990,Canon,Fotografía
3,Parlante Bluetooth Woburn,599990,Marshall,Audio
4,AirPods 2ª Generación Con Estuche De Carga,149990,Apple,Audio


In [14]:
x = 0
data_purchase = []

while (x < 5):
    date = pd.to_datetime('today').strftime('%Y-%m-%d %H:%M:%S')
    
    product = df['PRODUCT_NAME'][random.randint(0,14)]
    price = df[df['PRODUCT_NAME'] == product]['PRICING'].values[0]
    brand = df[df['PRODUCT_NAME'] == product]['BRAND'].values[0]
    category = df[df['PRODUCT_NAME'] == product]['CATEGORY'].values[0]
    source_choice = random.choice(source)
    pay = get_pay_method(source_choice, status_purchase, payment_online, payment_store)
    city = random.choice(cities)
    
    purchase = {
        'purchase_id': str(hashlib.sha256(f"{x} {product} {price} {date} {source_choice} {pay[1]}".encode('utf-8')).hexdigest()[:10]),
        'product_name' : product,
        'price': str(price),
        'Payment_Mehtod':pay[0],
        'Status' : pay[1],
        'Order_Type' : pay[2],
        'City': city,
        'Latitude' : str(get_coords(city)[0]) ,
        'Longitude' :  str(get_coords(city)[1]),
        'Source':source_choice,
        'Brand' : brand,
        'Category': category,
        'Created_at': date
    }
    
    data_purchase.append(pd.DataFrame(purchase, index=[x]))
    print(purchase)
    x+=1
    time.sleep(random.choice([1,2]))

{'purchase_id': 'b62cfc0e16', 'product_name': 'Apple iPhone 15 Pro 128GB', 'price': '1179990', 'Payment_Mehtod': 'Cash', 'Status': 'COMPLETED', 'Order_Type': 'STORE', 'City': 'Los Ángeles', 'Latitude': '-37.46827078074566', 'Longitude': '-72.35255427015483', 'Source': 'On-site', 'Brand': 'Apple', 'Category': 'Celulares y Teléfonos', 'Created_at': '2024-01-13 20:56:15'}
{'purchase_id': '66ef7d46d6', 'product_name': 'LED 50” AU7090 4K UHD Smart TV 2022', 'price': '309990', 'Payment_Mehtod': 'Debit_card', 'Status': 'FRAUD', 'Order_Type': 'ONLINE', 'City': 'Santiago', 'Latitude': '-33.41811053129037', 'Longitude': '-70.6069033367', 'Source': 'Online', 'Brand': 'Samsung', 'Category': 'TV y Video', 'Created_at': '2024-01-13 20:56:17'}
{'purchase_id': '9d3403646b', 'product_name': 'LED 50” AU7090 4K UHD Smart TV 2022', 'price': '309990', 'Payment_Mehtod': 'Debit_card', 'Status': 'COMPLETED', 'Order_Type': 'STORE', 'City': 'Concepción', 'Latitude': '-36.82574208790205', 'Longitude': '-73.04847

In [15]:
data_purchase[0]

,purchase_id,product_name,price,Payment_Mehtod,Status,Order_Type,City,Latitude,Longitude,Source,Brand,Category,Created_at
0,b62cfc0e16,Apple iPhone 15 Pro 128GB,1179990,Cash,COMPLETED,STORE,Los Ángeles,-37.46827078074566,-72.35255427015483,On-site,Apple,Celulares y Teléfonos,2024-01-13 20:56:15


In [16]:
df_final = pd.concat(data_purchase, ignore_index=True)
df_final

,purchase_id,product_name,price,Payment_Mehtod,Status,Order_Type,City,Latitude,Longitude,Source,Brand,Category,Created_at
0,b62cfc0e16,Apple iPhone 15 Pro 128GB,1179990,Cash,COMPLETED,STORE,Los Ángeles,-37.46827078074566,-72.35255427015483,On-site,Apple,Celulares y Teléfonos,2024-01-13 20:56:15
1,66ef7d46d6,LED 50” AU7090 4K UHD Smart TV 2022,309990,Debit_card,FRAUD,ONLINE,Santiago,-33.41811053129037,-70.6069033367,Online,Samsung,TV y Video,2024-01-13 20:56:17
2,9d3403646b,LED 50” AU7090 4K UHD Smart TV 2022,309990,Debit_card,COMPLETED,STORE,Concepción,-36.82574208790205,-73.0484756767045,On-site,Samsung,TV y Video,2024-01-13 20:56:18
3,17929d93b3,Celular Samsung Galaxy S23 Ultra 512GB,1199990,Cash,COMPLETED,STORE,Concepción,-36.82574208790205,-73.0484756767045,On-site,Samsung,Celulares y Teléfonos,2024-01-13 20:56:20
4,9d3dddd2c6,Mouse Gamer G203 Prodigy Logitech,19990,Credit_card,COMPLETED,STORE,Los Ángeles,-37.46827078074566,-72.35255427015483,On-site,Logitech,Computadores,2024-01-13 20:56:21
